In [14]:
import sys
sys.path.append('../')

import polars as pl
import numpy as np
from polang.polars_expression_parser import *



In [33]:
from pyparsing import Word, alphanums


expr_parser = ()

ops = polang("a")

# Calculations
operand = (
    ppc.number.set_parse_action(Floatingpoint)
    | QuotedString("'").set_parse_action(String)
    | Word(alphas).set_parse_action(Column)
)

function_name = Word(identchars, identbodychars)
function_body = Forward()
function_body <<= (
    function_name
    + Suppress("(")
    + delimited_list(operand, delim=",")
    + Suppress(")")
).set_parse_action(Function)

operand.parse_string("'3'")


ParseResults([String(3)], {})

In [8]:
expr_parser = get_infix_parser()

expr = "sin(a)"

parsed = expr_parser.parseString(expr)
plexpr = parsed[0].eval()
print(plexpr)

df = pl.DataFrame({"Time": np.linspace(0,1), "a": np.linspace(1,2), "b": np.linspace(2,3), "c": np.linspace(3,4)})
df.select([pl.all(), plexpr.alias(expr)])

col("a").sin()


Time,a,b,c,sin(a)
f64,f64,f64,f64,f64
0.0,1.0,2.0,3.0,0.841471
0.020408,1.020408,2.020408,3.020408,0.852322
0.040816,1.040816,2.040816,3.040816,0.862817
0.061224,1.061224,2.061224,3.061224,0.872953
0.081633,1.081633,2.081633,3.081633,0.882726
0.102041,1.102041,2.102041,3.102041,0.892131
0.122449,1.122449,2.122449,3.122449,0.901165
0.142857,1.142857,2.142857,3.142857,0.909823
0.163265,1.163265,2.163265,3.163265,0.918102


In [13]:
cosexpr = expr_parser.parseString("cos(a)")[0].eval()
derexpr = expr_parser.parseString("der sin(a)")[0].eval()
df.select([pl.all(), cosexpr.alias("cos"), derexpr.alias("dsin(a)/dt")])

Time,a,b,c,cos,dsin(a)/dt
f64,f64,f64,f64,f64,f64
0.0,1.0,2.0,3.0,0.540302,null
0.020408,1.020408,2.020408,3.020408,0.523018,0.522982
0.040816,1.040816,2.040816,3.040816,0.505516,0.505481
0.061224,1.061224,2.061224,3.061224,0.487804,0.4877
0.081633,1.081633,2.081633,3.081633,0.469888,0.469855
0.102041,1.102041,2.102041,3.102041,0.451776,0.451745
0.122449,1.122449,2.122449,3.122449,0.433477,0.433447
0.142857,1.142857,2.142857,3.142857,0.414997,0.414968
0.163265,1.163265,2.163265,3.163265,0.396344,0.396316
